# Introduction to the CrossClient

In this notebook you learn, how to connect to the CrossPlatform using the CrossClient
an to get a contract and validate your data against the contract. 

## Packages and data

In [1]:
from crosscontract import CrossClient

## Determine user

Here we assume that you have some
.env file that stores your credentials and we extract them from there.

In [3]:
from dotenv import load_dotenv
import os
load_dotenv(".env")
username = os.getenv("USER")
password = os.getenv("PASSWORD")

# we explicitly set the domain here as we want to connect to our staging instance
domain =  "https://backstage.sweetcross.link"

## Connect to the CrossPlatform

To connect to the platform using CrossClient you need a registered user. To create the
client, simply provide it the username and password. Here we assume that you have some
.env file that stores your credentials and we extract them from there.

In [ ]:
my_client = CrossClient(
    username=username, 
    password=password, 
    base_url=domain
)
my_client._base_url
df_overview = my_client.contracts.overview()

'https://backstage.sweetcross.link'

That's it. The platform knows who you are and how you want to login. So let's get 
a contract and use it for data validation.

## Getting an overview

First we want to get an overview which contracts are on the CrossPlatform and 
what they contain. For this we have the method `client.contracts.overview` that
provides a Pandas Dataframe with the metadata of the contract (as well as the status
of the contract).

In [6]:
df_overview = my_client.contracts.overview()

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'https://backstage.sweetcross.link/user/auth/login'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422

## Getting a contract

We want to get the contract that provides the values used for fuels in CROSS. It's
name is `dim_fuel`.

To get the contract we use `client.contracts.get`. This provides us a `ContractResource` which
essentially is a `CrossContract` that lives on the CrossPlatform. The `ContractResource`
is the central object to work with contracts and allows you to get, add, and delete
data for a contract. As the contract is 
saved on the platform, we are not allowed to change it.

In [ ]:
conract_name = ""
fuel_contract_resource = my_client.contracts.get(name=)

In [3]:
test_contract = {
    "name": "test_contract",
    "title": "Test Contract",
    "description": "A simple test contract",
    "valueFields": [{"field": "value"}],
    "timeFields": [{"field": "year", "frequency": "yearly"}],
    "locationFields": [{"field": "country", "locationType": "country"}],
    "primaryKey": ["year", "country"],
    "fields": [
        {
            "name": "value",
            "type": "number",
            "constraints": {
                "required": True,
                "minimum": 0.0,
                "maximum": 100.0,
                "unique": True,
            },
        },
        {
            "name": "year",
            "type": "integer",
            "constraints": {"required": True, "minimum": 2000, "maximum": 2025},
        },
        {
            "name": "country",
            "type": "string",
            "constraints": {"required": False, "maxLength": 6, "minLength": 2},
        },
    ],
}

df_test_data = pd.DataFrame(
    {
        "value": [10.0, 20.0, 30.0],
        "year": [2020, 2021, 2022],
        "country": ["US", "DE", "FR"],
    }
)

## Initialize the connection

Given the connection parameters, we initialize the client. Here we use the local 
client so ensure that all docker containers are running:

In [4]:
username = "jabmin"
password = "jabmin_password"
domain = "http://localhost"
client.init_client(username, password, base_url=domain)

Ensure that we start from a *clean* database, i.e., the test_contract does not
exists:

In [5]:
try:
    contracts.delete_data_table_for_contract("test_contract")
except Exception as e:
    print(f"Error deleting data-table:\n {e}")
try:
    contracts.delete_contract("test_contract")
except Exception as e:
    print(f"Error deleting contract:\n {e}")

Error deleting data-table:
 Failed to delete data table for contract 'test_contract': 409 {"detail":"Data contract with ID test_contract does not have a data table. [CrossFitBackend]"}


## Create contracts

Creating contracts is a two-step process:
1. Create a contract
2. Create a data table for the contract

The first steps checks whether the contract is valid and creates the placeholder, 
i.e., no other contract with the same name could be created as long as the contract
is registered.

The second step creates a data table based on the schema provided in the contract.
This data table allows to insert data into the database.

### Create and delete contracts

A contract can be created from file or from a given dictionary. For simplicity,
we use the dictionary variant here. The file variant would be exactly the same 
but we would pass the filename instead fo the dictionary.

In [6]:
my_contract = contracts.create_contract(test_contract)

If the contract was create successfully,  we get back a "DataContract" object. 
This is nothing else as the  object representation of the dictionary representation 
of the contract. So we can access all attributes of the contract using dot syntax.

In [7]:
print(my_contract)
print(f"Contract name: {my_contract.name}")
print(f"Contract fields: {my_contract.fields}")

name='test_contract' title='Test Contract' description='A simple test contract' tags=[] primaryKey=['year', 'country'] foreignKeys=None valueFields=[ValueFieldDescriptor(field='value')] timeFields=[TimeFieldDescriptor(field='year', frequency='yearly')] locationFields=[LocationFieldDescriptor(field='country', locationType='country')] fields=[NumberField(name='value', title=None, description=None, constraints=NumericConstraint[float](required=True, unique=True, minimum=0.0, maximum=100.0, enum=None), type='number'), IntegerField(name='year', title=None, description=None, constraints=NumericConstraint[int](required=True, unique=False, minimum=2000, maximum=2025, enum=None), type='integer'), StringField(name='country', title=None, description=None, constraints=StringConstraint(required=False, unique=False, pattern=None, minLength=2, maxLength=6, enum=None), type='string')]
Contract name: test_contract
Contract fields: [NumberField(name='value', title=None, description=None, constraints=Num

Once we created the contract, we cannot create a second contract with the same name:

In [8]:
try:
    my_contract = contracts.create_contract(test_contract)
except Exception as e:
    print(f"Error creating contract:\n {e}")

Error creating contract:
 Failed to create contract: 409 {"detail":"Data contract with name 'test_contract' already exists. [CrossFitBackend]"}


So we have to delete the contract first, before adding another contract with the
same name. If the contract does not exist, we get an error:

In [9]:
contracts.delete_contract(contract_name="test_contract")
try:
    contracts.delete_contract(contract_name="test_contract")
except Exception as e:
    print(f"Error deleting contract:\n {e}")

Error deleting contract:
 Failed to delete contract 'test_contract': 404 {"detail":"Data contract with ID test_contract not found. [CrossFitBackend]"}


### Create and delete data-tables

Data table are are created with the **create_data_table_for_contract** function.
If the corresponding contract does not exist, we get an error:

In [10]:
try:
    contracts.create_data_table_for_contract(contract_name="test_contract")
except Exception as e:
    print(f"Error creating data-table:\n {e}")

Error creating data-table:
 Failed to create data table for contract 'test_contract': 404 {"detail":"Data contract with ID test_contract not found. [CrossFitBackend]"}


So we create the contract first and then the data table

In [11]:
my_contract = contracts.create_contract(test_contract)
contracts.create_data_table_for_contract(contract_name=my_contract.name)

<Response [201 Created]>

As the data table now exists, we get an error if we try to create it a second 
time:

In [12]:
try:
    contracts.create_data_table_for_contract(contract_name="test_contract")
except Exception as e:
    print(f"Error creating data-table:\n {e}")

Error creating data-table:
 Failed to create data table for contract 'test_contract': 409 {"detail":"Data contract with ID test_contract already has a data table. [CrossFitBackend]"}


If we try to delete a contract for which a data contract exists, we get an error, 
as the no data-table can exist without a contract:

In [13]:
try:
    contracts.delete_contract(contract_name="test_contract")
except Exception as e:
    print(f"Error deleting contract:\n {e}")

Error deleting contract:
 Failed to delete contract 'test_contract': 409 {"detail":"Data contract with ID test_contract has an associateddata table and cannot be deleted. [CrossFitBackend]"}


So we first delete the data-table and then contract:

In [14]:
contracts.delete_data_table_for_contract(contract_name="test_contract")
contracts.delete_contract(contract_name="test_contract")

<Response [204 No Content]>

### For convenience: All in one

For convenience we have functions that allow us to do the two steps in one:

In [15]:
my_contract = contracts.create_contract_and_data_table(contract_data=test_contract)
my_contract

<Response [201 Created]>

In [16]:
contracts.delete_contract_and_data_table(contract_name="test_contract")

<Response [204 No Content]>

## Adding data

We cannot insert data, without have a valid contract and associated data table.
So let's create again our test contract with the data table:

In [17]:
my_contract = contracts.create_contract_and_data_table(contract_data=test_contract)

We insert data passing the contract name and the data in the form of a dataframe
to the **insert_data** function:

As we have a primary key constraint in the contract, we cannot add the data again.

In [18]:
try:
    data.insert_data(contract_name="test_contract", df=df_test_data)
except Exception as e:
    print(f"Error inserting data:\n {e}")

We can get back the data as dataframe using the **get_data** function with the
contract name:

In [19]:
df_out = data.get_data(contract_name="test_contract")
df_out

,value,year,country
0,10.0,2020,US
1,20.0,2021,DE
2,30.0,2022,FR


## Clean up

We clean by deleting the data table and the associated contract:

In [20]:
contracts.delete_data_table_for_contract(contract_name="test_contract")

<Response [204 No Content]>